In [5]:
!pip install --upgrade scipy nltk


In [3]:
!pip install streamlit pyngrok --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00


In [13]:
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
import random

# Ensure required NLTK resources are downloaded
def download_nltk_resources():
    # Added 'punkt_tab' to the list of resources to download
    resources = ['punkt', 'averaged_perceptron_tagger', 'wordnet', 'punkt_tab']
    for resource in resources:
        try:
            # Attempt to find the resource first to avoid unnecessary downloads
            # Adjusted the find path logic slightly for clarity and accuracy
            resource_path = None
            if resource == 'punkt' or resource == 'punkt_tab':
                resource_path = f'tokenizers/{resource}'
            elif resource == 'averaged_perceptron_tagger':
                resource_path = f'taggers/{resource}'
            elif resource == 'wordnet':
                 resource_path = f'corpora/{resource}'

            if resource_path:
                 nltk.data.find(resource_path)

        except LookupError:
            print(f"Downloading {resource}...")
            nltk.download(resource)

    # Explicitly download the English version of the tagger if it's still missing
    # This part seems less critical for the current error but kept for completeness
    try:
        nltk.data.find('taggers/averaged_perceptron_tagger_eng')
    except LookupError:
         print(f"Downloading averaged_perceptron_tagger_eng...")
         nltk.download('averaged_perceptron_tagger_eng')


download_nltk_resources()

def get_noun_keywords(text):
    words = word_tokenize(text)
    tagged_words = pos_tag(words)
    nouns = [word for word, pos in tagged_words if pos.startswith("NN") and len(word) > 3]
    return list(set(nouns))

def get_distractors(word):
    distractors = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            lemma_name = lemma.name().replace("_", " ")
            if lemma_name.lower() != word.lower():
                distractors.add(lemma_name)
        if len(distractors) >= 5:
            break
    return list(distractors)

def create_mcq(sentence, answer):
    question = sentence.replace(answer, "_______", 1)
    distractors = get_distractors(answer)
    distractors = random.sample(distractors, min(3, len(distractors)))
    options = distractors + [answer]
    random.shuffle(options)
    return question, options, answer

if __name__ == "__main__":
    input_text = """Japan is an island nation located in East Asia, known for its rich culture, advanced technology, and unique history. The country consists of four main islands—Honshu, Hokkaido, Kyushu, and Shikoku—and thousands of smaller ones. Japan has a population of over 125 million people and Tokyo, its capital, is one of the largest metropolitan areas in the world. Historically, Japan was ruled by samurai warriors and governed under the shogunate system for centuries, which lasted until the mid-19th century. The Meiji Restoration in 1868 marked a period of rapid modernization and industrialization, transforming Japan into a major global power. After devastating losses in World War II, Japan adopted a pacifist constitution and rebuilt its economy to become a leading force in electronics, automobiles, and robotics. Japan also maintains deep cultural traditions such as tea ceremonies, sumo wrestling, and festivals that blend Shinto and Buddhist influences, making it a fascinating blend of the old and new"""
    sentences = sent_tokenize(input_text)
    keywords = get_noun_keywords(input_text)
    used = set()
    questions = []

    for sent in sentences:
        for word in keywords:
            if word in sent and word not in used:
                question, options, answer = create_mcq(sent, word)
                questions.append((question, options, answer))
                used.add(word)
                if len(questions) == 3:
                    break
        if len(questions) == 3:
            break

    if questions:
        for i, (q, opts, ans) in enumerate(questions, 1):
            print(f"Q{i}: {q}")
            for idx, opt in enumerate(opts):
                print(f"  {chr(97+idx)}) {opt}")
            print(f"Answer: {ans}\n")
    else:
        print("Could not generate questions. Try a different paragraph.")

Q1: Japan is an island nation located in _______ Asia, known for its rich culture, advanced technology, and unique history.
  a) due east
  b) East
  c) Orient
  d) eastward
Answer: East

Q2: Japan is an _______ nation located in East Asia, known for its rich culture, advanced technology, and unique history.
  a) island
Answer: island

Q3: Japan is an island nation located in East Asia, known for its rich culture, advanced technology, and unique _______.
  a) account
  b) history
  c) chronicle
  d) story
Answer: history



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
!ngrok config add-authtoken 2tykSIV146iFJG5JKzBDlVp6osp_4yKYVXP9jkwAZRKkz39QZ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [1]:
!pkill ngrok

In [19]:
%%writefile app.py
import streamlit as st
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
import random

# Download necessary NLTK resources
def download_nltk_resources():
    resources = ['punkt', 'averaged_perceptron_tagger', 'wordnet']
    for resource in resources:
        try:
            nltk.data.find(f'tokenizers/{resource}' if resource == 'punkt' else f'taggers/{resource}' if resource == 'averaged_perceptron_tagger' else f'corpora/{resource}')
        except LookupError:
            nltk.download(resource)

download_nltk_resources()

# Extract important nouns from the text
def get_noun_keywords(text):
    words = word_tokenize(text)
    tagged_words = pos_tag(words)
    nouns = [word for word, pos in tagged_words if pos.startswith("NN") and len(word) > 3]
    return list(set(nouns))

# Get distractor words from WordNet
def get_distractors(word):
    distractors = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            lemma_name = lemma.name().replace("_", " ")
            if lemma_name.lower() != word.lower():
                distractors.add(lemma_name)
        if len(distractors) >= 5:
            break
    return list(distractors)

# Create an MCQ from a sentence and answer
def create_mcq(sentence, answer):
    question = sentence.replace(answer, "_______", 1)
    distractors = get_distractors(answer)
    distractors = random.sample(distractors, min(3, len(distractors)))
    options = distractors + [answer]
    random.shuffle(options)
    return question, options, answer

# Streamlit UI
st.title("🧠 MCQ Generator using NLTK and WordNet")

text_input = st.text_area("Enter a paragraph:", height=200)

if st.button("Generate MCQs"):
    if text_input:
        sentences = sent_tokenize(text_input)
        keywords = get_noun_keywords(text_input)
        used_keywords = set()
        questions = []

        for keyword in keywords:
            for sentence in sentences:
                if keyword in sentence:
                    question, options, answer = create_mcq(sentence, keyword)
                    if len(options) >= 2:  # ensure we have valid options
                        questions.append((question, options, answer))
                        used_keywords.add(keyword)
                    break  # go to next keyword after 1 question
            if len(questions) == 3:
                break

        if questions:
            for i, (q, opts, ans) in enumerate(questions, 1):
                st.markdown(f"**Q{i}: {q}**")
                for idx, opt in enumerate(opts):
                    st.write(f"{chr(97+idx)}) {opt}")
                st.success(f"✅ Answer: {ans}")
        else:
            st.warning("Could not generate enough unique questions. Try a longer or more diverse paragraph.")
    else:
        st.warning("Please enter a paragraph first.")


Overwriting app.py


In [21]:
from pyngrok import ngrok

# Kill any existing tunnels
ngrok.kill()

# Launch the app
!streamlit run app.py &> /dev/null &

# Get public URL
# Use 'addr' instead of 'port' to specify the address to connect to
public_url = ngrok.connect(addr=8501)
print(f"👉 Click here to open Streamlit app: {public_url}")

👉 Click here to open Streamlit app: NgrokTunnel: "https://e558-35-196-176-220.ngrok-free.app" -> "http://localhost:8501"
